### Imports

In [1]:
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
from Crypto.Hash import SHA3_256
from Crypto.PublicKey import DSA
from Crypto.Signature import DSS
from datetime import datetime, timedelta


### Constants

In [2]:
key_size = 256 // 8
salt_size = 128 // 8
giga_byte = 1024 * 1024 * 1024
dsa_key_size = 1024

plain_text_sizes = {
	"low": 16,
	"medium": 2048,
	"big": giga_byte
}

blocks_count = {
	"low": giga_byte // plain_text_sizes["low"],
	"medium": giga_byte // plain_text_sizes["medium"],
	"big": giga_byte // plain_text_sizes["big"]
}

blocks_count


{'low': 67108864, 'medium': 524288, 'big': 1}

### Data generation

In [3]:
def generate_data(key_size, salt_size, plain_text_block_size, blocks_count):
	keys = [get_random_bytes(key_size) for i in range(blocks_count)]
	salts = [get_random_bytes(salt_size) for i in range(blocks_count)]
	texts = [get_random_bytes(plain_text_block_size) for i in range(blocks_count)]

	return {'keys': keys, 'salts': salts, 'texts': texts}


# Cryptoprimitives

### AES (Symmetric cipher)

In [4]:
def test_AES(data, blocks_count):
	for index in range(blocks_count):
		cipher = AES.new(data['keys'][index], AES.MODE_CBC, iv=data['salts'][index])
		cipher.encrypt(data['texts'][index])


### SHA-3 (Hash function)

In [5]:
def test_SHA3(data, blocks_count):
    for index in range(blocks_count):
        hash_obj = SHA3_256.new()
        hash_obj.update(data['texts'][index])


### DSA (Public Key Signature)

In [6]:
def test_DSA(data, blocks_count):
    signer = DSS.new(data['dsa_key'], 'fips-186-3')

    for index in range(blocks_count):
        signer.sign(SHA3_256.new(data['texts'][index]))


# Time

In [7]:
iteration_count = 10

def test_time_function(func, data, need_dsa_key):
	def sub_function(*args):
		start_time = datetime.now()

		if need_dsa_key == True:
			data["dsa_key"] = DSA.generate(dsa_key_size)
		
		func(data, *args)
		
		return datetime.now() - start_time

	return sub_function

def test_block(func, data_size, need_dsa_key = False):
	def sub_function(*args):

		time = timedelta(0)
			
		for i in range(iteration_count):
			data = generate_data(key_size, salt_size, plain_text_sizes[data_size], blocks_count[data_size])

			time += test_time_function(func, data, need_dsa_key)(*args)
		
		print(f'{time.total_seconds() * 100}ms')

	return sub_function


## AES test

In [8]:
test_block(test_AES, "big")(blocks_count["big"])


1904.0337000000002ms


In [9]:
test_block(test_AES, "medium")(blocks_count["medium"])


5997.5101ms


In [10]:
test_block(test_AES, "low")(blocks_count["low"])


551916.578ms


## SHA-3 test

In [11]:
test_block(test_SHA3, "big")(blocks_count["big"])


2383.0948ms


In [12]:
test_block(test_SHA3, "medium")(blocks_count["medium"])


3862.8647ms


In [13]:
test_block(test_SHA3, "low")(blocks_count["low"])


180097.6085ms


## DSA test

In [14]:
test_block(test_DSA, "big", True)(blocks_count["big"])


2687.4157ms


In [15]:
test_block(test_DSA, "medium", True)(blocks_count["medium"])


155213.4733ms


In [16]:
test_block(test_DSA, "low", True)(blocks_count["low"])
